<a href="https://colab.research.google.com/github/fanyang2012/fanyang2012/blob/main/Data_Scraping_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scraping

Inhalte:

*   Preisinfo (UVP, Vorzugspreis, Grundpreis, Rabatt)
*   Lieferanten-SKU
*   Produktanzahl
*   Images (URLs, Bilderanzahl)
*   Produktbeschreibung (Basisinfo, Attribute)
*   Produktstatus

Webseiten: 

*   Shop-Apotheke.de
*   Mueller.de
*   Reuter.de
*   Otto.de


## Shop-apotheke.de & Mueller.de -> Kosmetik & Nahrungsmittel
### 1. Images

*   Marken, Kategorien
*   URLs der Produktbilder
*   Anzahl der Produktbilder
*   Definition des Namen des Images
*   base_image, additional_images
*   File_path - standard path + Ordnername (Kosmetik, Nahrungsmittel)
*   Save_Image -> File path + File name

### 2. Produktinfos
*  Produktpreis -> shop-apotheke.de
*  RPA -> Otto.de
*  Produktstatus -> Mueller.de, Reuter.de, Otto.de
*  Produktbeschreibung - html
*  Übersetzung - Produktbeschreibung
*  SEO: URL, Titel (Produktname), Keywords (Marken, Kategorie), Beschreibung (Kurzbeschreibung)
*  SKU-Nummer


## URL-Pinyin



*   https://www.cnblogs.com/diantong/p/12606328.html
*   https://pypi.org/project/xpinyin/
*   https://www.jb51.net/article/167461.htm



In [ ]:
!pip install xpinyin
!pip install jieba

In [ ]:
name = 'Grohe 壁挂式马桶套装无边框带白色马桶座圈'

In [ ]:
def pinyin_url(name): 

  from xpinyin import Pinyin # Package for transfering into pinyin
  import jieba # Package to cut chinese words

  name = name.replace(',', '').replace(' ', '') # remove comma in the chinese name
  names = jieba.lcut(name) # split chinese name into different words

  namen = ''
  for i in range(len(names)):
    namen = namen + names[i]
    if i+1 < len(names): 
      namen = namen + '-' # add '-' between words

  ret = p.get_pinyin(namen, '') # tranfer the name into pinyin

  return ret

In [ ]:
pinyin_url(name)

'Grohe-biguashi-matong-taozhuang-wu-biankuang-dai-baise-matong-zuoquan'

## 1. Install packages

In [ ]:
!pip install cloudscraper

## 2. Import packages

In [ ]:
import requests # requests html pages
import cloudscraper
import re # Regex
from bs4 import BeautifulSoup

In [ ]:
import os # os.join()
import sys

## 3. Data Scraping - Shop-Apotheke.de

In [ ]:
#from abc import abstractproperty
# get html text from the website
def apo_HTMLText(apo): 
    try: 
        r = requests.get(apo, timeout = 10)
        #r.raise_for_status()
        #r.encoding = r.apparent.encoding
        return r.text
    except: 
        return "Requests failure"

# get urls of all product images
def apo_ImageURL(html): 
    try: 
        all_url = []
        urls = re.findall(r'<li\sdata-index="\d*"><a\sdata-qa-id="form-gallery-image-\d*"\shref="https://cdn.shop-apotheke.com/images/[A-Z]\d*/\d*/\d*/[A-Z]\d*-[a-z]\d*.[a-z]*"', html)
        for i in range(len(urls)): 
            image_url = eval(urls[i].split('href=')[1])
            all_url.append(image_url)
        return all_url
    except: 
        print("getImageURL Failure")

# Produktstatus
def apo_produktstatus(html): 
  soup = BeautifulSoup(html, 'html.parser')
  layout = soup.find_all(id = 'layout')
  if 'Leider' or 'Außer Handel' or 'Derzeit nicht lieferbar' in str(layout[0]): 
    status = 'Disabled'
  if 'Sofort lieferbar' in str(layout[0]): 
    status = 'Enabled'
  return status

# Preisinformationen
def apo_priceinfo(html, status): 

  if status == 'Enabled': 

    soup = BeautifulSoup(html, 'html.parser')
    # Vorzugspreis
    if re.findall('Ihr\sPreis</span><div\sclass="a-h2\su-no-margin\su-no-wrap"><span\sclass="u-no-wrap\sa-Price">€\s\d*,\d*', html) == []:       
      vorzugspreis = ''
    else: 
      vorzugspreis = re.findall('Ihr\sPreis</span><div\sclass="a-h2\su-no-margin\su-no-wrap"><span\sclass="u-no-wrap\sa-Price">€\s\d*,\d*', html)[0]
      vorzugspreis = re.findall('\d*,\d*', vorzugspreis)[0].replace(',', '.')
      # print(vorzugspreis)

    # UVP
    if soup.find_all('span', "a-PriceOld--price") == []: 
      uvp = vorzugspreis
    else: 
      uvp = str(soup.find_all('span', "a-PriceOld--price")[0])
      uvp = re.findall('\d*,\d*', uvp)[0].replace(',', '.')
    ''' Or it can be defined as following: 
    if re.findall('<span\sclass="a-PriceOld--price"><span\sclass="u-no-wrap\sa-Price">[^\<]*', html) == []: 
      uvp = vorzugspreis
    else:
      uvp = re.findall('<span\sclass="a-PriceOld--price"><span\sclass="u-no-wrap\sa-Price">€\s\d*,\d*', html)[0]
      uvp = re.findall('\d*,\d*', uvp)[0]
    '''
    # print(uvp)

    # Grundpreis
    grundpreis = str(soup.find_all('div', "a-h6 u-no-margin u-font-weight--normal u-no-wrap o-ProductVariant--base-price")[0])
    if grundpreis.split('>')[-2].split('<')[0] == '': 
      grundpreis = ''
    else: 
      grundpreis = re.findall('\d*,\d*\s€\s\/\s\d*\sml', grundpreis)[0].replace('\xa0', ' ')
    # print(grundpreis)

    # Rabatt
    rabatt = str(soup.find_all('span', "a-CircleBadge__inner")[0])
    if re.findall('\d*%', rabatt) == []: 
      rabatt = ''
    else: 
      rabatt = re.findall('\d*%', rabatt)[0]
    '''
    rabatt = re.findall('<span\sclass="a-CircleBadge__inner">[^\<]*', html)[0]
    if re.findall('\d*%', rabatt) == []: 
      rabatt = ''
    else: 
      rabatt = re.findall('\d*%', rabatt)[0]
    '''
    # print(rabatt)

  if status == 'Disabled': 
    uvp = 0
    vorzugspreis = 0
    rabatt = 0
    grundpreis = 0

  return [uvp, vorzugspreis, grundpreis, rabatt]

# Produktbeschreibung
def apo_Produktdetails(html): 
  soup = BeautifulSoup(html, 'html.parser')
  details = str(soup.find_all(id="o-ProductAdditionalInformation__teaser")[0])
  return details

# Lieferanten-SKU
def apo_sku(apo): 
  sku = re.findall(r'\d{5,}', apo)[0]
  return sku

# save the images with the defined file names in the specific paths
def saveImage(html, save_path, file_name): 
    try: 
        completeName = os.path.join(save_path, file_name) # completeName of file
        print(completeName)
        image_data = requests.get(html).content
        file = open(completeName, "wb") # file using  which translate the byte format into needed format
        file.write(image_data) # write content with byte format into file
        #file.close()
    except: 
        print("saveImage Failure")

# implement the above mentioned functions
if __name__ == '__main__': 
  #  apo = 'https://www.shop-apotheke.com/arzneimittel/1173889/abtei-gelenk-aktiv-1500.htm?eventName=click%20on%20product%20list%20item&eventType=click&objectIDs=%5b01173889%5d&position=1&query=abtei%20gelenk&queryID=93f77a95f41f57c06d590e63b20078d2'
    apo = 'https://www.shop-apotheke.com/arzneimittel/571748/grippostad-c-hartkapseln.htm'
   # apo = 'https://www.shop-apotheke.com/beauty/9433002/dr-hauschka-reinigungsmilch.htm?query=hauschk&queryID=9eda5a881846f1bbb73012e6e6269039&objectIDs=[09433002]&position=1&eventName=click%20on%20product%20in%20suggest&eventType=click'
    html = apo_HTMLText(apo)
    print(apo_ImageURL(html))
    print('The number of images: {}'.format (len(apo_ImageURL(html))))
    productstatus = apo_produktstatus(html)
    print(productstatus)
    print(apo_priceinfo(html, productstatus))
    print(apo_Produktdetails(html))
    print(apo_sku(apo))

In [ ]:
status = re.findall('<p\sclass="u-font-weight--bold\su-no-margin\su-color--\w*">[^\<]*', html)

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
str(soup.find_all(id="o-ProductAdditionalInformation__teaser")[0])

'<div class="a-truncate-with-fade-out" data-clientside-hook="scrollableTables" data-scrollable-classname="o-ProductAdditionalInformation__scrollable" id="o-ProductAdditionalInformation__teaser"><div><p><strong>SCHNELLE LINDERUNG VON ERKÄLTUNGSSYMPTOMEN MIT DER NR. 1 UNTER DEN ERKÄLTUNGSMITTELN*</strong><br/>\n  Erkältungen sind meist Virusinfektionen der Atemwege, die sich auf den ganzen Körper auswirken können. Viele Erkältete leiden bei einer Erkältung gleichzeitig an Schnupfen, Kopf- und Gliederschmerzen, Husten und Fieber. Alle diese Erkältungsbeschwerden gemeinsam können mit einem Kombi-Präparat bekämpft werden. Grippostad® C Kapseln sind Deutschlands meistverwendete Erkältungsmedizin bei Erkältungskrankheiten und grippalen Infekten. Sie enthalten eine Kombination aus Paracetamol, Chlorphenaminmaleat, Vitamin C und Coffein. Sie wirken gegen Reizhusten, Erkältungsfieber, Kopf- und Gliederschmerzen und Schnupfen in einem und lindern schnell diese gemeinsam<br/>\n  auftretenden Erkäl

## 4. Data Scraping - Mueller.de

In [ ]:
def mueller_HTMLText(mueller): 
    try: 
        r = requests.get(mueller, timeout = 10)
        #r.raise_for_status()
        #r.encoding = r.apparent.encoding
        return r.text
    except: 
        return "Requests failure"

# Alle Links der Produktbilder
def mueller_ImageURL(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        basic = soup.find_all('img',"mu-product-gallery__placeholder-image") # Basic images URL
        img_basic = str(basic[0]).split('"')[-2] # Extract the image url
        pdmain = re.findall(r'\d*-\w*-\d*\\/pdmain', html) # Extract core parts of other images
        
        images = []
        for ele in pdmain: 
            core = ele.replace('\\', '') # delete \\
            image = re.sub('\d*-\w*-\d*/pdmain', core, img_basic) # replace the core parts
            images.append(image) # Get URLs of all images
        return images
    except: 
        return "mueller_ImageURL failure"

# Produktstatus
def mueller_produktstatus(html): 
  if 'Die aufgerufene Seite konnte leider nicht gefunden werden' in html: 
    status = 'Disabled'
  else:
    status = 'Enabled'
  return status

#Preisinformationen
def mueller_priceinfo(html): 

  soup = BeautifulSoup(html, 'html.parser')

  if soup.find_all('span', 'mu-product-price__price mu-product-price__price--promo') == []: 
    uvp = str(soup.find_all('span', "mu-product-price__price ")[0]).split('>')[-2].replace(' ', '').replace('\n', '').split('€')[0]
    vorzugspreis = uvp
    rabatt = ''
  else: 
    uvp = str(soup.find_all('span', "mu-product-price__price mu-product-price__price--original")[0]).split('>')[-2].replace(' ', '').replace('\n', '').split('€')[0]
    vorzugspreis = str(soup.find_all('span', "mu-product-price__price mu-product-price__price--promo")[0]).split('>')[-2].replace(' ', '').replace('\n', '').split('€')[0]
    rabatt = str(soup.find_all('div', "mu-product-price__percentage-saving")[0])
    rabatt = re.findall(r'\d*%', rabatt)[0]

  grundpreis = str(soup.find_all('div', "mu-product-price__additional-info")[0]).split('>')[-2]
  if re.findall(r'\d*,\d*\s€\s\/\s\d*\s\w*', grundpreis) == []: 
    grundpreis = ''
  else: 
    grundpreis = re.findall(r'\d*,\d*\s€\s\/\s\d*\s\w*', grundpreis)[0]

  return [uvp, vorzugspreis, grundpreis, rabatt]

# Produktbeschreibung
def mueller_Details(html): 

    soup = BeautifulSoup(html, 'html.parser')
    ### output parts of "Artikelbeschreibung" from html
    detail = soup.find_all(id = "details")
    table = soup.find_all(id = "features")
    
    details = str(detail[0]) + '<p></p>' + str(table[0])
    
    ### Convert the uppercase´substrings into lowercase
    
    attr = ['Inhaltsstoffe', 'Zutaten']
    
    for word in attr: 
        if word in details: 
            part_before = details.split(word + '</td>')[0]
            lowerpart = str.lower(details.split(word + '</td>')[-1].split('</td>')[0])
            part_after = details.split(word + '</td>')[-1].split('</td>')[1:]
            
            part_after_merge = ''
            for i in range(len(part_after)): 
                part_after_merge = part_after_merge + part_after[i] + '</td>'
            details = part_before + word + '</td>' + lowerpart + '</td>' + part_after_merge
            
    ### add parts of "Zutaten" if there exists
    
    if 'id="nutrition"' in html: 
        nutrition = soup.find_all(id = 'nutrition')
        description = details + '<p></p>' + str(nutrition[0])
    
    return description

# Lieferanten_SKU
def mueller_sku(html): 

    soup = BeautifulSoup(html, 'html.parser')
    sku = str(soup.find_all('div', "mu-product-details-page__article-number")[0])
    sku = re.findall('\d{4,}', sku)[0]
    return sku

if __name__ == '__main__': 
    
    url = 'https://www.mueller.de/p/home-by-berner-obstkorb-bonbons-2748089/'
    html = mueller_HTMLText(url)
    urls = mueller_ImageURL(html)
    description = mueller_Details(html)
    print(mueller_priceinfo(html))
    print(mueller_sku(html))
    print(len(urls))
    print(urls)
    print(description)

## 5. Data Scraping - Reuter.de

### Offene Punkte: Produktstatus

In [ ]:
reuter = 'https://www.reuter.de/grohe-eurosmart-einhand-spueltischbatterie-energie-spar-funktion-a516584.php'

In [ ]:
reuter = 'https://www.reuter.de/grohe-minta-kuechenarmatur-mit-ausziehbarer-dual-spuelbrause-chrom-a555626.php'

In [ ]:
def reuter_HTMLText(reuter): 
  try: 
    scraper = cloudscraper.create_scraper()
    html = scraper.get(reuter).text
    return html
  except: 
    print('reuter_HTMLText failure')

'''
pip install selenium
pip install webdriver-manager
# selenium 3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

# Get source code
driver.get(url)
html = driver.page_source
time.sleep(2)
print(html)

# close web browser
browser.close()
'''

def reuter_produktstatus(html): 
  if 'leider nicht gefunden' in html: 
    status = 'Disabled'
  else: 
    status = 'Enabled'
  return status

# Alle Links der Produktbilder
def reuter_ImageURL(html): 
     
    try: 
        urls = re.findall(r'https://img.reuter.de/products/fg/fg-\w+?.+\w', html)
        return urls
    except: 
        print("reuter_ImageURL Failure")

# Produktbeschreibung
def reuter_description(html): 
  soup = BeautifulSoup(html, 'html.parser')
  detail = str(soup.find_all("e2-details-block")[0])
  details = '<h1>Produktdetails</h1><p></p>' + detail.replace('<dl class="specification-list -light column">','<br></br><br></br>').replace('<div class="_pseudo-h2">','<p></p><h2>').replace('', '')
  description = re.sub('\\n<br></br><br></br>\\n<dt\sclass="specification-list\s-attribute">Artikelnummer:<\/dt>\\n<dd\sclass="specification-list\s-value">\d*<\/dd>\\n', '', details)
  return description

# Preisinformation
def reuter_price(html): 
  price_info = re.findall('"item":{"unit":"STÜCK","quantity":\d*,"price":\d*.\d*,"retail":\d*.\d*,"default":\d*.\d*,"difference":{"value":\d*.\d*,"percentage":\d*}}', html)[0]
  uvp = re.findall('"retail":\d*.\d*', price_info)[0].split(":")[1]
  vorzugspreis = re.findall('"price":\d*.\d*', price_info)[0].split(":")[1]
  rabatt = re.findall('"percentage":\d*', price_info)[0].split(":")[1]
  return [uvp, vorzugspreis, rabatt]

# Produktanzahl
def reuter_produktanzahl(html):
  produktanzahl = re.findall('\d* Stück', html)[0].split(' ')[0]
  return produktanzahl

# Lieferanten-SKU
def reuter_sku(html): 
  sku = re.findall('Artikelnummer:</dt>\\n<dd\sclass="specification-list\s-value">\d*', html)[0].split('>')[-1]
  return sku

## 6. Data Scraping - Otto.de

### Offene Punkte: Produktstatus

In [ ]:
def otto_HTMLText(otto): 
  r = requests.get(otto)
  html = r.text
  return html

# Alle Links der Produktbilder
def otto_ImageURL(html): 
  all_url = re.findall('data-image-url="https:\/\/i.otto.de\/i\/otto\/[^\\\/\"]*', html)
  img_url = []
  for i in all_url: 
    url = i.split('="')[1]
    img_url.append(url)
  img_url = list(set(img_url)) # aviod the duplicate urls
  return img_url

# Preisinformationen
def otto_prices(html): 
  soup = BeautifulSoup(html, 'html.parser')
  price_info = soup.find_all('div', "prd_price js_prd_price")
  uvp = re.findall('<span\sid="oldPriceAmount">\d*.\d*,\d*',str(price_info[0]))[0].split('>')[-1]
  vorzugspreis = re.findall('id="reducedPriceAmount"\sitemprop="price">\d*.\d*,\d*',str(price_info[0]))[0].split('>')[-1]
  rabatt = re.findall('\d*%',str(price_info[0]))[0]
  return [uvp, vorzugspreis, rabatt]

# Lieferanten-SKU
def otto_sku(html): 
  sku = re.findall('content="sku:\w*', html)[0].split(':')[-1]
  return sku

# Produktbeschreibungen
def otto_details(html):
   soup = BeautifulSoup(html, 'html.parser')
   part1 = str(soup.find_all('div',"prd_module prd_module--noLine prd_sellingPoints js_prd_sellingPoints")[0])
   part3_before = soup.find_all('div', "prd_moreBox__content js_prd_moreBox__content")[0]
   part3 = str(part3_before).replace('<table class="dv_characteristicsTable"><caption>', '<p></p><b2><strong>').replace('</caption>',':</strong></b2><p></p>').replace('<td class="left"><span>', '<br></br>').replace('</span></td><td>', ':  ')
   if 'prd_moreBox__content pl_copy100 js_prd_moreBox__content' in html: 
     part2 = str(soup.find_all('div', "prd_moreBox__content pl_copy100 js_prd_moreBox__content")[0])
   else: 
     teaser = soup.find_all('div', "rc_teaser__text")
     part2 = ''
     for i in teaser: 
       part2 = part2 + str(i)

   description = '<h2><strong>Artikelbeschreibung</strong></h2><br></br>' + part1 + '<br></br>' + part2 + '<p></p><h2><strong>Details</strong></h2>' + part3
   return description


In [ ]:
otto = 'https://www.otto.de/p/wmf-wasserkocher-bueno-1-7-l-2400-w-cromargan-C319409656/#variationId=318465537'